## Kohler - Multi-input Categorical Embeddings with Numericals

### Loading Necessary Files and Libraries

In [1]:
import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Input, Dense, LSTM
from keras.preprocessing.text import Tokenizer
from keras import layers
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras import optimizers
import pickle
import os
from keras.layers import initializers 
from keras.layers import regularizers 
from keras.layers import constraints 
from keras.layers import Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.constraints import max_norm
from keras.layers import LeakyReLU
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.layers import MaxPooling2D, Flatten, Conv2D
from keras.utils import to_categorical
initializer1 = keras.initializers.RandomNormal
initializer2 = keras.initializers.Zeros()
initializer3 = keras.initializers.glorot_uniform(seed=None)
initializer4 = keras.initializers.lecun_normal(seed=None)
%matplotlib inline

Using TensorFlow backend.


## Load Data

In [3]:
data = pd.read_csv("cardio.csv")
data.drop("key", inplace=True, axis=1)
print(data.shape)

(1831, 22)


In [4]:
data.head(1)

,X.1,X.2,X.3,X.4,X.5,X.6,X.7,X.8,X.9,X.10,...,X.13,X.14,X.15,X.16,X.17,X.18,X.19,X.20,X.21,target
0,0.004912,0.693191,-0.20364,0.595322,0.35319,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0


In [5]:
data["target"].value_counts()

0    1655
1     176
Name: target, dtype: int64

In [6]:
all = data
positive = data[data["target"] > 0]
negative = data[data["target"] == 0]

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(1831, 22) (176, 22) (1655, 22) 1831


In [7]:
all.drop("target", inplace=True, axis=1)
positive.drop("target", inplace=True, axis=1)
negative.drop("target", inplace=True, axis=1)

print(all.shape, positive.shape, negative.shape, positive.shape[0] + negative.shape[0])

(1831, 21) (176, 21) (1655, 21) 1831


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Positive is the anomalies - Train AutoEncoder with Negatives

In [8]:
INPUT_SIZE = negative.shape[1]
AUTO_ENCODER_SHAPE = 128

In [9]:
train_size = int(negative.shape[0]*90/100)
negative = negative.sample(frac=1)
X_train = negative.iloc[:train_size]
X_test = negative.iloc[-(negative.shape[0] - train_size):]

# force the model to lower the autoencoder reconstruction cost
Y = np.zeros([X_train.shape[0], 1], dtype = float) # ZEROS MATRIX with shape (,1)

print(X_train.shape, X_test.shape, Y.shape, INPUT_SIZE, X_train.shape[0] + X_test.shape[0])

(1489, 21) (166, 21) (1489, 1) 21 1655


## Build the Encoder

In [10]:
input = Input(shape=(INPUT_SIZE,))

input_encoder = layers.Dense(AUTO_ENCODER_SHAPE, activation = 'relu')(input)
hidden2 = layers.Dense(64, activation = 'relu')(input_encoder)
encoded = layers.Dense(32, activation = 'relu')(hidden2)

Instructions for updating:
Colocations handled automatically by placer.


## Build the Decoder

In [11]:
hidden3 = Dense(64, activation='relu')(encoded)
output_decoder = Dense(AUTO_ENCODER_SHAPE, activation='relu')(hidden3)
# decoded = Dense(100, activation='relu')(output_decoder)

In [12]:
from keras import backend as K
from keras.layers import Layer

class ReconstructionCostLayer(Layer):

    def __init__(self,output_dim):
        self.output_dim = output_dim
        super(ReconstructionCostLayer, self).__init__()

    def build(self, input_shape):
        super(ReconstructionCostLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        left = K.slice(x, [0, 0], [-1, AUTO_ENCODER_SHAPE])
        right = K.slice(x, [0, AUTO_ENCODER_SHAPE], [-1, -1])
        return K.sum(K.square(left - right), axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

### Concatenate Encoder input and Decoder output

In [13]:
input_encoder_plus_output_decoder = layers.concatenate([input_encoder, output_decoder], axis=1)


### Call the custom layer

In [14]:
reconstruction_cost_layer = ReconstructionCostLayer(1)(input_encoder_plus_output_decoder)

# Build the AutoEncoder with Regression Model

In [15]:
model = Model(input, reconstruction_cost_layer)

#### Visualize Model Architecture

In [16]:
adam = optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-4, decay=0.0, amsgrad=True)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mse'])


In [17]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 21)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          2816        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           2080        dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

# Train

In [18]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', 
                                         histogram_freq=0, 
                                         write_graph=True, 
                                         write_images=True)

history = model.fit(X_train, Y, epochs=400, batch_size=50, 
                    verbose=1,
                    shuffle=True,
                    callbacks=[tbCallBack],
                   )  

Instructions for updating:
Use tf.cast instead.
Epoch 1/400
1489/1489 [==============================] - 0s 111us/step - loss: 73.6769 - mean_squared_error: 73.6769
Epoch 2/400
1489/1489 [==============================] - 0s 25us/step - loss: 6.0815 - mean_squared_error: 6.0815
Epoch 3/400
1489/1489 [==============================] - 0s 26us/step - loss: 2.1467 - mean_squared_error: 2.1467
Epoch 4/400
1489/1489 [==============================] - 0s 25us/step - loss: 1.1407 - mean_squared_error: 1.1407
Epoch 5/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.6923 - mean_squared_error: 0.6923
Epoch 6/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.4513 - mean_squared_error: 0.4513
Epoch 7/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.3303 - mean_squared_error: 0.3303
Epoch 8/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.2593 - mean_squared_error: 0.2593
Epoch 9/400
1489/1489 [==============

1489/1489 [==============================] - 0s 25us/step - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 71/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 72/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 73/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0069 - mean_squared_error: 0.0069
Epoch 74/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 75/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 76/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0066 - mean_squared_error: 0.0066
Epoch 77/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0064 - mean_squared_error: 0.0064
Epoch 78/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0063 - mean_

Epoch 141/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 142/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 143/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 144/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 145/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 146/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 147/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 148/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 149/400
1489/1489 [==============================] - 0s 24us/step 

1489/1489 [==============================] - 0s 25us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 212/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 213/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 214/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 215/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 216/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 217/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 218/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 219/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0018

1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 282/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 283/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013   
Epoch 284/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013   
Epoch 285/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 286/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 287/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 288/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 289/400
1489/1489 [==============================] - 0s 26us/step - loss: 

1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010   
Epoch 352/400
1489/1489 [==============================] - 0s 26us/step - loss: 0.0010 - mean_squared_error: 0.0010
Epoch 353/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010
Epoch 354/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010   
Epoch 355/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010
Epoch 356/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010   
Epoch 357/400
1489/1489 [==============================] - 0s 25us/step - loss: 0.0010 - mean_squared_error: 0.0010   
Epoch 358/400
1489/1489 [==============================] - 0s 24us/step - loss: 0.0010 - mean_squared_error: 0.0010   
Epoch 359/400
1489/1489 [==============================] - 0s 25us/step

# Get Negative (Trained) Reconstruction Costs

In [19]:
reconstruction_costs = model.predict(X_train)

In [20]:
print("Count: {}  Mean: {}  Std: {}  Min: {}  Mx: {}"\
      .format(len(reconstruction_costs),
                  reconstruction_costs.mean(),
                  reconstruction_costs.std(),
                  reconstruction_costs.min(),
                  reconstruction_costs.max()))

Count: 1489  Mean: 0.02540929615497589  Std: 0.01602231152355671  Min: 0.00017033368931151927  Mx: 0.1046205386519432


In [21]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Negative (New) Reconstruction Costs

In [22]:
reconstruction_costs = model.predict(X_test)

In [23]:
print("Count: {}  Mean: {}  Std: {}  Min: {}  Mx: {}"\
      .format(len(reconstruction_costs),
                  reconstruction_costs.mean(),
                  reconstruction_costs.std(),
                  reconstruction_costs.min(),
                  reconstruction_costs.max()))

Count: 166  Mean: 3.1236469745635986  Std: 39.704593658447266  Min: 0.0033745993860065937  Mx: 513.1383056640625


In [24]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))

# Get Positive Reconstruction Costs

In [25]:
reconstruction_costs = model.predict(positive)

In [26]:
print("Count: {}  Mean: {}  Std: {}  Min: {}  Mx: {}"\
      .format(len(reconstruction_costs),
                  reconstruction_costs.mean(),
                  reconstruction_costs.std(),
                  reconstruction_costs.min(),
                  reconstruction_costs.max()))

Count: 176  Mean: 29.014619827270508  Std: 153.52224731445312  Min: 0.03623105585575104  Mx: 1197.669921875


In [27]:
# unique, counts = np.unique(reconstruction_costs, return_counts=True)
# dict(zip(unique, counts))